<a href="https://colab.research.google.com/github/feiyoung/ReadPapers/blob/master/pytorch_code1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the VAE model
class VAE(nn.Module):
    def __init__(self, input_size=784, hidden_size=256, latent_size=20):
        super(VAE, self).__init__()

        # Encoder
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc_mu = nn.Linear(hidden_size, latent_size)
        self.fc_logvar = nn.Linear(hidden_size, latent_size)

        # Decoder
        self.fc2 = nn.Linear(latent_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, input_size)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        z = F.relu(self.fc2(z))
        reconstruction = torch.sigmoid(self.fc3(z))
        return reconstruction

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # KL divergence
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

# Training function
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

# Test function
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))



In [8]:
os.getcwd()

'/content'

# 新段落

In [2]:
# Hyperparameters
batch_size = 128
epochs = 10
log_interval = 10
learning_rate = 1e-3

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_loader = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform),
                          batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(datasets.MNIST('./data', train=False, download=True, transform=transform),
                         batch_size=batch_size, shuffle=False, num_workers=2)

# Initialize the model and optimizer
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



100%|██████████| 9912422/9912422 [00:00<00:00, 430203340.62it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 84415117.65it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 172953494.80it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 15203933.57it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
train_loader

In [7]:

# Training loop
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 138.170868
Train Epoch: 1 [1280/60000 (2%)]	Loss: 142.223419
Train Epoch: 1 [2560/60000 (4%)]	Loss: 128.787292
Train Epoch: 1 [3840/60000 (6%)]	Loss: 135.288803
Train Epoch: 1 [5120/60000 (9%)]	Loss: 131.010178
Train Epoch: 1 [6400/60000 (11%)]	Loss: 132.853302
Train Epoch: 1 [7680/60000 (13%)]	Loss: 129.935104
Train Epoch: 1 [8960/60000 (15%)]	Loss: 131.650436
Train Epoch: 1 [10240/60000 (17%)]	Loss: 134.125381
Train Epoch: 1 [11520/60000 (19%)]	Loss: 127.018242
Train Epoch: 1 [12800/60000 (21%)]	Loss: 133.727219
Train Epoch: 1 [14080/60000 (23%)]	Loss: 133.742264
Train Epoch: 1 [15360/60000 (26%)]	Loss: 125.585449
Train Epoch: 1 [16640/60000 (28%)]	Loss: 126.842140
Train Epoch: 1 [17920/60000 (30%)]	Loss: 131.505554
Train Epoch: 1 [19200/60000 (32%)]	Loss: 131.441681
Train Epoch: 1 [20480/60000 (34%)]	Loss: 135.054642
Train Epoch: 1 [21760/60000 (36%)]	Loss: 125.035583
Train Epoch: 1 [23040/60000 (38%)]	Loss: 133.775604
Train Epoch: 1 [24320/60000 